# Funkcionalno programiranje

Amer Hasanović

<style>
pre {
    overflow: auto !important;
}
</style>

## Ekstenzije tipova

**Svi** `F#` tipovi podržavaju ekstenzije u formi metoda, funkcija koje:

* mogu biti pozivane na vrijednostima, tj. instancama tipova,
* mogu biti pozivane na samim tipovima, u slučaju da su metodi definirani kao **static**.

In [185]:
module Complex =
  type T = { Real : float ; Imag : float } with 
    member this.Print = $"{this.Real}+{this.Imag}i"
  
  let create r i = { Real = r; Imag = i}

let t = Complex.create 5 8
t.Print

5+8i

In [186]:
module Complex =
  type T = { Real : float ; Imag : float } with 
    member this.Print = $"{this.Real}+{this.Imag}i"
  
  let create r i = { Real = r; Imag = i}

  type T with
    member this.AltPrint = $"{this.Real}+j{this.Imag}" 

let t = Complex.create 5 8
t.AltPrint

5+j8

In [187]:
module Complex =
  type T = { Real : float ; Imag : float } with 
    member x.Print = $"{x.Real}+{x.Imag}i"
  
  let create r i = { Real = r; Imag = i}


module Drugi =
  type Complex.T with
    member this.AltPrint = $"{this.Real}+j{this.Imag}" 
    
  let t = Complex.create 5 8
  t.AltPrint |> printfn "%s"

5+j8


In [188]:
module Complex =
  type T = { Real : float ; Imag : float } with 
    member this.Print = $"{this.Real}+{this.Imag}i"
  
  let create r i = { Real = r; Imag = i}


module Drugi =
  type Complex.T with
    member this.AltPrint = $"{this.Real}+j{this.Imag}" 
    
module Treci =
  let t = Complex.create 5 8
  t.AltPrint |> printfn "%s"

Error: input.fsx (14,5)-(14,13) typecheck error The type 'T' does not define the field, constructor or member 'AltPrint'. Maybe you want one of the following:
   Print

In [189]:
module Complex =
  type T = { Real : float ; Imag : float } with 
    member this.Print = $"{this.Real}+{this.Imag}i"
  
  let create r i = { Real = r; Imag = i}


module Drugi =
  type Complex.T with
    member this.AltPrint = $"{this.Real}+j{this.Imag}" 
    

module Treci =
  let t = Complex.create 5 8
  open Drugi
  t.AltPrint |> printfn "%s"

5+j8


In [190]:
module FooBar =
  type System.Double with
    member this.ToComplex = Complex.create this 0
  
  type System.Decimal with
    member this.ToComplex = Complex.create (double this) 0
  
  let f : float = 3.3
  let d  = 5.5M
  f.ToComplex.Print |> printfn "%s"
  d.ToComplex.Print |> printfn "%s"


3.3+0i
5.5+0i


In [191]:
module Complex =
  type T = { Real : float ; Imag : float }

  let create r i = { Real = r; Imag = i}
  let print {T.Real = r; Imag = i}  = $"{r}+{i}*i"

  type T with
    member this.Print = print this
    static member create  = create 

(Complex.T.create 5 4).Print |> printfn "%s"

5+4*i


In [192]:
let foo c = "Complex broj je:" + c.Print

Error: input.fsx (1,34)-(1,41) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.

In [193]:
let foo (c : Complex.T) = "Complex broj je: " + c.Print

Complex.create 3 2 |> foo

Complex broj je: 3+2*i

In [194]:
let bar c = "Complex broj je: " +  (Complex.print c)

Complex.create 3 2 |> bar

Complex broj je: 3+2*i

In [195]:
module Complex =
  type T = { Real : float ; Imag : float } with
   override this.ToString () = 
     sprintf "%.3f+%.3fi" this.Real this.Imag

  let create r i = { Real = r; Imag = i}
  let print c = c.ToString()
  let saberi (c : T) r i = { Real = c.Real + r; Imag = c.Imag + i }
  let oduzmi (c : T) r i = { Real = c.Real - r; Imag = c.Imag - i }

  type T with
    member this.Saberi r i = saberi this r i
    member this.Oduzmi (dc:T) = oduzmi this dc.Real dc.Imag
    member this.Oduzmi (r, ?i) = 
      match i with 
        | None -> oduzmi this r 0
        | Some i -> oduzmi this r i
    static member (-) (a:T, b:T) = a.Oduzmi(b) 
    static member (-) (a:T, b:float) = a.Oduzmi(b)


In [196]:
let c1 = Complex.create 5 5
let c2 = Complex.create 10 10

In [197]:
c1.Saberi 8 5 |> printfn "%O"


13.000+10.000i


In [198]:
c1.Oduzmi(i=2, r=5) |> printfn "%O"
c1.Oduzmi(5) |> printfn "%O"
c1.Oduzmi(c2) |> printfn "%O"

0.000+3.000i
0.000+5.000i
-5.000+-5.000i


In [199]:
open type Complex.T
c1 - c2 |> Complex.print |> printfn "%s"
c1 - 3.1 |> Complex.print |> printfn "%s"

-5.000+-5.000i
1.900+5.000i


## `F#` imperativno programiranje

In [200]:
let mutable a = 5
a |> printfn "%d" 

5


In [201]:
a <- 10
a |> printfn "%d" 

10


In [202]:
let foo (r : byref<int>) =
  printfn $"foo je dobio {r}"
  r <- r + 2

In [203]:
let mutable b = 10
foo &b
b |> printfn "%d"

foo je dobio 10
12


In [204]:
let bar(r : inref<int>) =
  printfn $"foo je dobio {r}"
  r <- r + 2


Error: input.fsx (3,3)-(3,13) typecheck error The byref pointer is readonly, so this write is not permitted.

In [205]:
let tar(r : outref<int>) =
  r <- r + 2


In [206]:
let sumWhile n =
  let mutable s = 0
  let mutable i = 0
  while i <= 10 do
    s <- i + s
    i <- i + 1
  s

sumWhile 5 |> printfn "%d" 

55


In [207]:
let sumFor n =
  let mutable s = 0
  for i in [1..n] do
    s <- s + i
  s

5 |> sumFor |> printfn "%d" 

15


In [208]:
type Student( name : string, last_name : string, dob : DateTime )  =
  static let mutable cnt = 0
  static do 
    printfn "Running only once, when the class is loaded!"

  do
    printfn $"Constructed a student {name}"
    cnt <- cnt + 1
    
  let cur_cnt = cnt
  let mutable l_n = last_name


  member this.Name = name
  member this.Dob = dob
  member this.Age = ( DateTime.Now - dob ).Days / 365
  member this.Id = cur_cnt
  
  member this.Hello(a,b) = $"{this.Name} says hello {a} {b}"

  member this.LastName 
   with get() = l_n
   and set(ln) = l_n <- ln

  member val Index = "" with get, set

  override this.ToString() = $"{name} {last_name} {dob}"

  new(n) = Student(n, "Anonymous", DateTime(2000,1,1))
  new() = Student("Foobar", "Tarbar", DateTime(2003,1,1))




Running only once, when the class is loaded!


In [209]:
let a = Student("Meho")
let b = Student()
let c = Student(last_name="Doe", dob=DateTime.Now, name="John")

Constructed a student Meho
Constructed a student Foobar
Constructed a student John


In [210]:
a.LastName |> printfn "%s"
b.LastName |> printfn "%s"
a.LastName <- "Mehić"
a.LastName |> printfn "%s"

Anonymous
Tarbar
Mehić


In [211]:
a.Age |> printfn "%d"
a.Id |> printfn "%d"
b.Id |> printfn "%d"

23
1
2


In [212]:
a.Hello(b="foo",a=10)

Meho says hello 10 foo

In [213]:
a |> printfn "%A"

Meho Anonymous 1/1/2000 12:00:00 AM


In [214]:
type Vector(x: float, y : float) =
  member this.x = x
  member this.y = y
  static member (~-) (v : Vector) =
    Vector(-1.0 * v.x, -1.0 * v.y)
  static member (*) (v : Vector, a) =
    Vector(a * v.x, a * v.y)
  static member (*) (a, v: Vector) =
    Vector(a * v.x, a * v.y)
  override this.ToString() =
    this.x.ToString() + " " + this.y.ToString()


In [215]:
let v1 = Vector(1.0, 2.0)

let v2 = v1 * 2.0
let v3 = 2.0 * v1

let v4 = - v2

printfn "%A" v1
printfn "%A" v2
printfn "%A" v3
printfn "%A" v4

1 2
2 4
2 4
-2 -4


In [216]:
type C(a: 'a, b: 'b) =
  let z = a
  let y = b
  member this.Foo(x: 'a) = printfn "%A %A %A" x y z

In [217]:
C(2, "foobar").Foo(5.5)

5.5 "foobar" 2


In [218]:
type Base(x: int) =
    let mutable z = x 
    do
        for i in 1..z do
            printf "Base: %d " i
        printf "\n"


type Derived(y: int) =
    inherit Base(y * 2)
    do
        for i in 1..y do
            printf "Derived: %d " i

In [219]:
Derived(2) |> ignore

Base: 1 Base: 2 Base: 3 Base: 4 
Derived: 1 Derived: 2 

In [220]:
[<AbstractClass>]
type Shape2D(x0: float, y0: float) =
    let mutable x, y = x0, y0
    let mutable rotAngle = 0.0

    member this.CenterX
        with get () = x
        and set xval = x <- xval

    member this.CenterY
        with get () = y
        and set yval = y <- yval

    abstract Area: float with get
    abstract Perimeter: float with get
    abstract Name: string with get

    member this.Move dx dy =
        x <- x + dx
        y <- y + dy

    abstract member Rotate: float -> unit
    default this.Rotate(angle) = rotAngle <- rotAngle + angle


In [221]:
type Square(x, y, sideLengthIn) =
    inherit Shape2D(x, y)
    member this.SideLength = sideLengthIn
    
    override this.Area = this.SideLength * this.SideLength
    override this.Perimeter = this.SideLength * 4.
    override this.Name = "Square"

In [222]:
type Circle(x, y, radius) =
    inherit Shape2D(x, y)
    let PI = 3.141592654
    member this.Radius = radius

    override this.Area = PI * this.Radius * this.Radius
    override this.Perimeter = 2. * PI * this.Radius
    override this.Name = "Circle"

    override this.Rotate(_) = ()

In [223]:
let square1 = Square(0.0, 0.0, 10.0)
let circle1 = Circle(0.0, 0.0, 5.0)
circle1.CenterX <- 1.0
circle1.CenterY <- -2.0
square1.Move -1.0 2.0
square1.Rotate 45.0
circle1.Rotate 45.0
let shapeList: list<Shape2D> = [ square1; circle1 ]
List.iter (fun (elem: Shape2D) -> printf " %f" elem.Area ) shapeList

 100.000000 78.539816

In [224]:
type IPrintable =
    abstract member Print: unit -> unit

type SomeClass1(x: int, y: float) =
    interface IPrintable with
        member this.Print() = printfn "%d %f" x y


In [225]:
SomeClass1(2, 3.3).Print()

Error: input.fsx (1,20)-(1,25) typecheck error The type 'SomeClass1' does not define the field, constructor or member 'Print'.

In [226]:
(SomeClass1(2, 3.3) :> IPrintable).Print()

2 3.300000


In [227]:
let makePrintable (x: int, y: float) =
    { new IPrintable with
        member this.Print() = printfn "%d %f" x y }

let x3 = makePrintable (1, 2.0)
x3.Print()

1 2.000000


In [228]:
let foo (x : IPrintable ) = x.Print()
SomeClass1(5, 4.5) |> foo 

5 4.500000


In [229]:
type Interface1 =
    abstract member Method1: int -> int

type Interface2 =
    abstract member Method2: int -> int

type Interface3 = 
  inherit Interface1
  inherit Interface2
  abstract member Method3: int -> int

type FooBar() =
  interface Interface3 with 
    member this.Method1 x = 2*x
    member this.Method2 x = x*x
    member this.Method3 x = x / 5
